In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

print(torchvision.__version__)
print(torch.__version__)

0.2.1

0.4.1

In [2]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "./hymenoptera_data/hymenoptera_data"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [3]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch,num_epochs-1))

        # Each epoch has a training and validation
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss =0.0
            running_corrects = 0

            # Iterate over data
            for inputs,labels in dataloaders[phase]:
                inputs = inputs.to(device)
                label = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track histoty if only in train
                with torch.set_grad_enabled(phase=='train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    # mode we calculate the loss by summing the final output and the auxiliary output
                    # but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        outputs,aux_outputs = model(inputs)
                        loss1 = criterion(outputs,label)
                        loss2 = criterion(aux_outputs,label)
                        loss = loss1+0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs,labels)

                    _,preds = torch.max(outputs,1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == label.data)

            epoch_loss = running_loss/len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double()/len(dataloaders[phase].dataset)

            print('{} loss {:.4f} Acc {:.4f}'.format(phase,epoch_loss,epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc

SyntaxError: invalid syntax (<ipython-input-3-24f3e36a4540>, line 62)